### Imports

In [1]:
# some imports we will need in this example
import os
import requests, zipfile
import numpy as np
import mudata as md
import scanpy as sc

import multiDGD
from multiDGD.utils import *

# make sure we are in the correct directory
def switch_to_project_dir(name):
    cwd = os.getcwd()
    dir_hierarchy = cwd.split('/')
    index = dir_hierarchy.index(name)
    if index != len(dir_hierarchy) - 1:
        os.chdir('/'.join(dir_hierarchy[:index+1]))
        print('Moved to project directory: ', os.getcwd())

switch_to_project_dir('multiDGD')

Moved to project directory:  /Users/vschuste/Documents/work_KU/projects/multiDGD


/opt/anaconda3/envs/omics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data loading and preparation

Mudata objects can be used in this model just like an anndata object. Mudata objects are actually even easier to use, since the modality feature assignment does not have to be specified with the `modality_key` as for anndata objects.

In [ ]:
# Download
data_dir = 'data/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
os.chdir(data_dir)
file_name = 'mouse_gastrulation.h5mu.zip'
file_url = 'https://api.figshare.com/v2/articles/23796198/files/41740323'

if not os.path.exists(os.path.join(data_dir, file_name)):
    print('Downloading data...')
    file_response = requests.get(file_url).json()
    file_download_url = file_response['download_url']
    response = requests.get(file_download_url, stream=True)
    with open(file_name, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# Unzip
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('.')
os.remove(file_name)
os.chdir('..')

In [2]:
# load some test data

# this is an example mudata object
data = md.read("./data/mouse_gastrulation.h5mu", backed=False)
data = data[::5,:]
# set it up with the model (similar to MultiVI)
# but we support both anndata and mudata objects
data = multiDGD.functions._data.setup_data(data, observable_key='celltype', covariate_keys=['stage'])

/opt/anaconda3/envs/omics/lib/python3.10/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/opt/anaconda3/envs/omics/lib/python3.10/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/opt/anaconda3/envs/omics/lib/python3.10/site-packages/multiDGD/functions/_data.py:71: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  data.obs['observable'] = data.obs[observable_k

Otherwise everyhing is the same as for modeling anndata objects.